In [ ]:
%reload_kedro

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_spine = catalog.load("spine_labeled")
df_mt_multic = pd.concat([catalog.load("master_table_train_multic"), catalog.load("master_table_test_multic")])
df_window_nbr = pd.concat([catalog.load("window_nbr_lookup_train_multic"), catalog.load("window_nbr_lookup_test_multic")])

In [ ]:
df_mt = df_mt_multic[["window_nbr"]] \
                    .merge(df_window_nbr, on="window_nbr", how="inner")
assert df_mt.shape[0] == df_mt_multic.shape[0]

In [ ]:
df_mt.head(2)

In [ ]:
df = df_mt.merge(df_spine[["open_time", "close_time", "target_time", "logret_cumsum"]],
                on=["open_time", "close_time", "target_time"],
                how="inner")
assert df.shape[0] == df_mt.shape[0]

In [ ]:
df.head()

In [ ]:
df.close_time.min(), df.close_time.max()

In [ ]:
df.logret_cumsum.min(), df.logret_cumsum.max()

In [ ]:
df.loc[:, "pctchg_cumsum"] = np.exp(df["logret_cumsum"]) - 1
df.pctchg_cumsum.min(), df.pctchg_cumsum.max()

In [ ]:
df.loc[:, "value_diff"] = (df["pctchg_cumsum"] / df["logret_cumsum"] - 1)*100

In [ ]:
plot = df["value_diff"].plot()
plt.title("Value difference between log-returns and returns")
plt.xlabel("Time index"); plt.ylabel("Difference (%) of values")

fig = plot.get_figure()
fig.savefig("/Users/gpalazzo/Desktop/fig_logret_pctchg_value_diff.png")

In [ ]:
print(f"Min values diff: {round((df.pctchg_cumsum.min() / df.logret_cumsum.min() - 1) * 100, 2)}%")

In [ ]:
print(f"Max values diff: {round((df.pctchg_cumsum.max() / df.logret_cumsum.max() - 1) * 100, 2)}%")